In [1]:
setwd("/Users/rebecca/sudmant/analyses/myotis/analysis/exploratory/species_peaks")

library(GSA)
library(dplyr)
library(qvalue)
library(data.table)

source("/Users/rebecca/sudmant/analyses/myotis/code/fisher_test.R")

shared_genes <- fread("results/data/shared_genes.csv", data.table = FALSE)


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



Attaching package: 'data.table'


The following objects are masked from 'package:dplyr':

    between, first, last




In [ ]:
## Load in Broad gene sets:
gsc <- GSA.read.gmt("/Users/rebecca/sudmant/analyses/myotis/data/genesets/msigdb_v2023.2.Hs_GMTs/msigdb.v2023.2.Hs.symbols.gmt")
sets <- gsc$genesets
names(sets) <- gsc$geneset.names

## Shared peaks

In [3]:
shared_peaks <- fread("results/data/shared_promoters_shared_peaks.csv", data.table = FALSE)
shared_peak_enrichments <- sort(unlist(lapply(sets, fisher_test,
                                              mod = shared_peaks$Gene,
                                              all = shared_genes$Gene)))
shared_peak_enrich_df <- data.frame(Set = names(shared_peak_enrichments), 
                                    Pval = shared_peak_enrichments,
                                    row.names = NULL)
shared_peak_enrich_df$Qval <- qvalue(shared_peak_enrich_df$Pval)$qvalues
head(shared_peak_enrich_df)

,Set,Pval,Qval
,<chr>,<dbl>,<dbl>
1,GOMF_STEROL_ESTERASE_ACTIVITY,6.439896e-05,1
2,REACTOME_DNA_DAMAGE_TELOMERE_STRESS_INDUCED_SENESCENCE,1.047510e-04,1
3,GOMF_NEUROTRANSMITTER_RECEPTOR_REGULATOR_ACTIVITY,2.239841e-04,1
4,HP_PARIETAL_FORAMINA,2.980208e-04,1
5,REACTOME_COPI_MEDIATED_ANTEROGRADE_TRANSPORT,3.555364e-04,1
6,GSE1740_UNSTIM_VS_IFNA_STIMULATED_MCSF_DERIVED_MACROPHAGE_DN,3.609137e-04,1


### No significant enrichments in shared peaks

## Semi-shared peaks

In [5]:
semi_shared_peaks <- fread("results/data/shared_promoters_semi_shared_peaks.csv", data.table = FALSE)

species <- unique(semi_shared_peaks$Species)
                                                                                 
missing_peak_enrichments <- lapply(species, function(spec){
  
  missing_peaks_spec <- semi_shared_peaks[semi_shared_peaks$Species %in% spec,]
  missing_peaks_enrich <- sort(unlist(lapply(sets, fisher_test,
                                             mod = missing_peaks_spec$Gene,
                                             all = shared_genes$Gene)))
  return(data.frame(Species = spec,
                    Set = names(missing_peaks_enrich), 
                    Pval = missing_peaks_enrich,
                    Qval = qvalue(missing_peaks_enrich)$qvalues,
                    row.names = NULL))
  
})
names(missing_peak_enrichments) <- species

In [6]:
tapply(semi_shared_peaks$Gene, semi_shared_peaks$Species, "[")

$`Myotis-Auriculus`
[1] "COX8A" "RPLP1"

$`Myotis-Californicus`
 [1] "ADAM9"    "CFHR4"    "IRF4"     "NPHS1"    "PDPR"     "PPIA"    
 [7] "PPP4R1"   "PRPS2"    "RRBP1"    "S100A7"   "SLC25A24" "TMED9"   

$`Myotis-Evotis`
[1] "CTRB1"    "DDX54"    "POLR2B"   "SERPINB1" "ZNF296"  

$`Myotis-Lucifugus`
[1] "ACTL7B" "CCT5"   "CD46"   "H2AC19"

$`Myotis-Occultus`
[1] "ATP5MC2" "MROH1"   "SETMAR"  "SLC35G1"

$`Myotis-Thysanodes`
[1] "ALOX12"  "DOC2B"   "PLK3"    "PPP1R2"  "TFCP2L1" "TRIM47" 

$`Myotis-Velifer`
[1] "VNN1"

$`Myotis-Volans`
[1] "CBR1"   "CCNYL1" "FABP3"  "MKRN1"  "TAAR1"  "TSSK1B"

$`Myotis-Yumanensis`
[1] "GNG5"     "GPKOW"    "HERC5"    "HSPA1A"   "IRGC"     "RAP2C"    "SLBP"    
[8] "SLC7A6OS" "SPEF1"

In [7]:
lapply(missing_peak_enrichments, head, 10)

,Species,Set,Pval,Qval
,<chr>,<chr>,<dbl>,<dbl>
1,Myotis-Lucifugus,GOBP_SINGLE_FERTILIZATION,0.0004596155,1
2,Myotis-Lucifugus,LI_AMPLIFIED_IN_LUNG_CANCER,0.0006543979,1
3,Myotis-Lucifugus,GOBP_FERTILIZATION,0.0006662110,1
4,Myotis-Lucifugus,GSE17721_CPG_VS_GARDIQUIMOD_2H_BMDC_UP,0.0008558799,1
5,Myotis-Lucifugus,GSE17721_PAM3CSK4_VS_CPG_8H_BMDC_UP,0.0010688411,1
6,Myotis-Lucifugus,HP_DECREASED_MEAN_CORPUSCULAR_HEMOGLOBIN_CONCENTRATION,0.0011416610,1
7,Myotis-Lucifugus,HP_PLACENTAL_ABRUPTION,0.0011416610,1
8,Myotis-Lucifugus,WEBER_METHYLATED_ICP_IN_SPERM_UP,0.0015219974,1
9,Myotis-Lucifugus,KEGG_MEDICUS_REFERENCE_ALTERNATIVE_PATHWAY_OF_COMPLEMENT_CASCADE_C3B_BREAKDOWN,0.0015219974,1


### No significant enrichments in N-1 peaks

## Private peaks

In [10]:
private_peaks <- fread("results/data/shared_promoters_private_peaks.csv", data.table = FALSE)
                                                                                  
private_peak_enrichments <- lapply(species, function(spec){
  
  private_peaks_spec <- private_peaks[private_peaks$Species %in% spec,]
  private_peaks_enrich <- sort(unlist(lapply(sets, fisher_test,
                                             mod = private_peaks_spec$Gene,
                                             all = shared_genes$Gene)))
  return(data.frame(Species = spec,
                    Set = names(private_peaks_enrich), 
                    Pval = private_peaks_enrich,
                    Qval = qvalue(private_peaks_enrich)$qvalues,
                    row.names = NULL))
  
})
names(private_peak_enrichments) <- species

In [11]:
tapply(private_peaks$Gene, private_peaks$Species, "[")

$`Myotis-Auriculus`
 [1] "ADH6"    "GK"      "GOLPH3"  "H2AZ1"   "HMX3"    "KLK9"    "OR51V1" 
 [8] "PABPC4"  "PLP2"    "RUVBL1"  "STRA6"   "TMEM127" "ZC3H3"  

$`Myotis-Californicus`
[1] "ARFGEF1" "EIF4A3" 

$`Myotis-Evotis`
 [1] "CLXN"      "CP"        "EIF3G"     "KRTAP13-2" "OR51A7"    "OR51C1P"  
 [7] "PKM"       "PRKRA"     "SIRPA"     "SRP14"     "TOMM22"    "TRUB2"    

$`Myotis-Lucifugus`
 [1] "ATP8B2"    "BSG"       "BTBD18"    "BTNL2"     "CHIA"      "CLCA1"    
 [7] "DDX60"     "DTD2"      "EDC3"      "EIF2S2"    "ENPP7"     "FAU"      
[13] "FLT1"      "GARIN3"    "GPBP1L1"   "GPS2"      "HNRNPH1"   "IVL"      
[19] "MANEAL"    "MCM3AP"    "MED4"      "MIA3"      "MRPS12"    "NEK2"     
[25] "NUP62"     "OR52B2"    "OR56A1"    "PACRG"     "PARP12"    "PRELID3B" 
[31] "RANBP3"    "RELT"      "TAAR2"     "TAF1"      "TARM1"     "TAS2R16"  
[37] "TMPRSS11D" "TRRAP"     "TTC7B"     "ZNF207"   

$`Myotis-Occultus`
 [1] "AKR1D1"  "C4BPA"   "CDK2AP1" "CELA2A"  "CKAP2"   "FAM169A" "FHL1"   
 [8] "MRPS33"  "NCL"     "RHOA"    "TMEM233" "UBQLN1"  "WTAP"    "ZBED6"  

$`Myotis-Thysanodes`
 [1] "ARG2"    "BAAT"    "C1D"     "CYTIP"   "DMTN"    "KIF4A"   "LYPLA1" 
 [8] "MSTN"    "MTMR1"   "NACC1"   "OBSCN"   "PRSS55"  "PRSS8"   "SDAD1"  
[15] "SGF29"   "SINHCAF" "SRSF3"   "UBE2V1" 

$`Myotis-Velifer`
 [1] "ABCA13"    "ALDOA"     "ATP1A1"    "BPNT2"     "CDC34"     "CNBD2"    
 [7] "COMMD1"    "DYNLT1"    "GOLPH3L"   "H2AZ2"     "HINT1"     "ISCU"     
[13] "KLRF1"     "KRTAP16-1" "LAGE3"     "MUC4"      "MYBL2"     "MYH3"     
[19] "NLRP1"     "NR1D2"     "PSMD7"     "PTPRZ1"    "RNF213"    "RPL6"     
[25] "RPS28"     "RPS7"      "RPS8"      "SF3B2"     "SPRYD4"    "TBC1D7"   
[31] "TMED10"    "TMEM199"   "TRIP11"    "UCK2"      "WASHC3"    "WRNIP1"   
[37] "ZBTB8OS"  

$`Myotis-Volans`
 [1] "ADAL"    "ADRM1"   "ANKRD17" "ARSF"    "ATP2B4"  "CACNA1H" "CHFR"   
 [8] "CMA1"    "FADS2"   "FBXO22"  "FPR2"    "FTMT"    "HNRNPDL" "HTR3C"  
[15] "IGBP1"   "MPC1L"   "MTCP1"   "NDUFA3"  "OR52D1"  "RPS29"   "RPS3"   
[22] "SLC6A4"  "TBCB"   

$`Myotis-Yumanensis`
[1] "ADPRHL1" "AOC3"    "KLHL15"  "METTL22" "SARDH"   "ZC3H11A"

In [12]:
lapply(private_peak_enrichments, head, 10)

,Species,Set,Pval,Qval
,<chr>,<chr>,<dbl>,<dbl>
1,Myotis-Lucifugus,GOBP_CENTROSOME_SEPARATION,0.0003899389,1
2,Myotis-Lucifugus,KEGG_OLFACTORY_TRANSDUCTION,0.0006074201,1
3,Myotis-Lucifugus,HP_ABNORMAL_NASOLACRIMAL_SYSTEM_MORPHOLOGY,0.0006074201,1
4,Myotis-Lucifugus,BOSCO_INTERFERON_INDUCED_ANTIVIRAL_MODULE,0.0008848400,1
5,Myotis-Lucifugus,GSE24081_CONTROLLER_VS_PROGRESSOR_HIV_SPECIFIC_CD8_TCELL_DN,0.0009181104,1
6,Myotis-Lucifugus,GSE25123_WT_VS_PPARG_KO_MACROPHAGE_IL4_AND_ROSIGLITAZONE_STIM_UP,0.0009483047,1
7,Myotis-Lucifugus,GOCC_NUCLEAR_PORE,0.0009924642,1
8,Myotis-Lucifugus,GSE16450_CTRL_VS_IFNA_12H_STIM_MATURE_NEURON_CELL_LINE_UP,0.0010761123,1
9,Myotis-Lucifugus,BIOCARTA_VEGF_PATHWAY,0.0012491120,1


## Semi-private peaks

In [23]:
tapply(private_peaks$Gene, private_peaks$Species, "[")

$`Myotis-Auriculus, Myotis-Californicus`
[1] "ASRGL1"  "EHBP1L1" "PNO1"   

$`Myotis-Auriculus, Myotis-Evotis`
[1] "JDP2"    "MAGEA10" "SF3A2"   "ZSCAN5B"

$`Myotis-Auriculus, Myotis-Lucifugus`
[1] "GMCL1"   "HSPA8"   "PEBP1"   "TAS2R42" "U2AF1"   "ZC3H4"  

$`Myotis-Auriculus, Myotis-Occultus`
[1] "ADCY10" "CYP4F3"

$`Myotis-Auriculus, Myotis-Thysanodes`
[1] "DEFB1" "RBMX" 

$`Myotis-Auriculus, Myotis-Velifer`
[1] "H1-8"     "LYRM1"    "OR52B4"   "QRICH2"   "SNRNP200"

$`Myotis-Auriculus, Myotis-Volans`
[1] "CFH"     "FAM151A" "MIDEAS"  "SRSF1"   "YBX3"   

$`Myotis-Auriculus, Myotis-Yumanensis`
[1] "RPL38"

$`Myotis-Californicus, Myotis-Lucifugus`
[1] "ERH"    "OR52K1"

$`Myotis-Californicus, Myotis-Thysanodes`
[1] "ZWINT"

$`Myotis-Californicus, Myotis-Velifer`
[1] "C2orf78" "STEEP1" 

$`Myotis-Californicus, Myotis-Volans`
[1] "C19orf53"

$`Myotis-Evotis, Myotis-Lucifugus`
[1] "DTX2"   "OR52H1" "RNASE1" "UBTFL1"

$`Myotis-Evotis, Myotis-Occultus`
[1] "FLG2"    "TBX1"    "TMPRSS9"

$`Myotis-Evotis, Myotis-Thysanodes`
[1] "FSIP2"   "GBP1"    "IMP3"    "NECTIN2" "NUTF2"  

$`Myotis-Evotis, Myotis-Velifer`
[1] "AMACR"   "CD300LB" "IGSF3"   "KCTD9"   "LRRC57" 

$`Myotis-Evotis, Myotis-Volans`
[1] "ALPI"    "LRRC37B"

$`Myotis-Evotis, Myotis-Yumanensis`
[1] "ADH1C" "FZD5"  "PPM1A"

$`Myotis-Lucifugus, Myotis-Occultus`
[1] "CPSF1"   "CYP1A1"  "NDE1"    "OR52E5"  "PICALM"  "PRELID1"

$`Myotis-Lucifugus, Myotis-Thysanodes`
 [1] "ABCA3"   "ATP5F1A" "CYP2B6"  "FBXL3"   "NAP1L1"  "R3HDM2"  "REG3G"  
 [8] "TMEM89"  "TSGA10"  "VARS1"  

$`Myotis-Lucifugus, Myotis-Velifer`
[1] "DEFB116" "G3BP1"   "HDAC2"   "MRPL27"  "QDPR"    "RPL14"   "RPL29"  
[8] "RPS2"    "TXN"    

$`Myotis-Lucifugus, Myotis-Volans`
 [1] "BIN2"     "C1orf43"  "KCTD12"   "LRRC37A2" "MOXD1"    "MYADM"   
 [7] "NLE1"     "NOP53"    "OR51I2"   "PRC1"     "SNW1"     "SSU72"   
[13] "TFAP2C"  

$`Myotis-Lucifugus, Myotis-Yumanensis`
[1] "CLEC12B" "S100A10"

$`Myotis-Occultus, Myotis-Thysanodes`
[1] "ALDH9A1" "FAM47E"  "GLTP"    "SYCN"   

$`Myotis-Occultus, Myotis-Velifer`
[1] "ACSL1"     "C20orf173" "H2AC1"     "KRT17"     "P2RY2"     "PTCHD1"   
[7] "RPL28"     "RPS13"     "ZFAND2B"  

$`Myotis-Occultus, Myotis-Volans`
[1] "CTSG"  "GALM"  "PRPF6"

$`Myotis-Occultus, Myotis-Yumanensis`
[1] "PRSS22"

$`Myotis-Thysanodes, Myotis-Velifer`
[1] "DOCK2" "RPL18" "WDR7" 

$`Myotis-Thysanodes, Myotis-Volans`
[1] "AP3B2"   "DUSP18"  "POU2F1"  "RNF113A" "SDR42E1" "WBP11"   "ZCRB1"  

$`Myotis-Velifer, Myotis-Volans`
[1] "CD8B2"    "DECR2"    "OAS1"     "TRAPPC12" "ZP1"     

$`Myotis-Velifer, Myotis-Yumanensis`
[1] "DCLK2"   "NDRG1"   "PLD6"    "SLC35F2" "SMG5"    "SYNE1"   "UBE2C"  

$`Myotis-Volans, Myotis-Yumanensis`
[1] "MXRA5"

In [21]:
private_peaks <- fread("results/data/shared_promoters_semi_private_peaks.csv", data.table = FALSE)

all_pairs <- table(private_peaks$Species)

species <- names(all_pairs)[all_pairs >= 3]

species <- species                        
private_peak_enrichments <- lapply(species, function(spec){
  
  private_peaks_spec <- private_peaks[private_peaks$Species %in% spec,]
  private_peaks_enrich <- sort(unlist(lapply(sets, fisher_test,
                                             mod = private_peaks_spec$Gene,
                                             all = shared_genes$Gene)))
  return(data.frame(Species = spec,
                    Set = names(private_peaks_enrich), 
                    Pval = private_peaks_enrich,
                    Qval = qvalue(private_peaks_enrich)$qvalues,
                    row.names = NULL))
  
})
names(private_peak_enrichments) <- species

In [22]:
lapply(private_peak_enrichments, head, 10)

,Species,Set,Pval,Qval
,<chr>,<chr>,<dbl>,<dbl>
1,"Myotis-Auriculus, Myotis-Californicus",HE_LIM_SUN_FETAL_LUNG_C3_VENOUS_ENDOTHELIAL_CELL,7.183165e-05,1
2,"Myotis-Auriculus, Myotis-Californicus",GSE22886_TH1_VS_TH2_12H_ACT_UP,2.781502e-04,1
3,"Myotis-Auriculus, Myotis-Californicus",GSE14415_INDUCED_TREG_VS_FAILED_INDUCED_TREG_DN,3.475427e-04,1
4,"Myotis-Auriculus, Myotis-Californicus",GSE14699_DELETIONAL_TOLERANCE_VS_ACTIVATED_CD8_TCELL_UP,3.851115e-04,1
5,"Myotis-Auriculus, Myotis-Californicus",GSE16450_CTRL_VS_IFNA_12H_STIM_MATURE_NEURON_CELL_LINE_DN,4.659832e-04,1
6,"Myotis-Auriculus, Myotis-Californicus",GSE17721_LPS_VS_POLYIC_12H_BMDC_UP,4.730671e-04,1
7,"Myotis-Auriculus, Myotis-Californicus",GSE17721_LPS_VS_POLYIC_8H_BMDC_UP,5.241378e-04,1
8,"Myotis-Auriculus, Myotis-Californicus",REACTOME_PHENYLALANINE_METABOLISM,1.141661e-03,1
9,"Myotis-Auriculus, Myotis-Californicus",ADCYAP1_TARGET_GENES,1.141661e-03,1
